# C964 Computer Science Capstone
## Machine Learning Project
### Jeff Fenwick

In [14]:
# install...
# wheel
# pandas
# pyarrow
# ipywidgets
# matplotlib
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
dataset = pd.read_csv("ds_salaries.csv")

In [34]:
for i in range(0, len(dataset.index)):
    print(dataset.iloc[i, 7])

79833
260000
109024
20000
150000
72000
190000
35735
135000
125000
51321
40481
39916
87000
85000
8000
41689
114047
5707
56000
43331
6072
47899
98000
115000
325000
42000
33511
100000
117104
59303
70000
68428
450000
46759
74130
103000
250000
10000
138000
45760
79833
50180
106000
112872
15966
76958
188000
105000
70139
6072
91000
45896
54742
60000
148261
38776
118000
120000
138350
110000
130800
21669
412000
45618
62726
49268
190200
105000
91237
62726
42197
82528
150000
235000
53192
100000
5409
270000
80000
79197
140000
54238
47282
153667
28476
59102
110000
170000
80000
88654
76833
19609
276000
29751
89294
12000
450000
70000
95746
75000
150000
36259
62000
73000
51519
187442
115000
150000
30428
94564
113476
103160
12000
45391
225000
50000
40189
90000
200000
60000
200000
50000
110037
10354
151000
120000
9466
20000
40570
100000
49646
38400
24000
100000
90000
63711
77364
220000
80000
135000
240000
150000
82500
100000
82744
62649
90000
153000
160000
168000
150000
75774
13400
144000
127221
119059
